In [1]:
import pandas as pd
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import snowflake 

df = None
with snowflake.connector.connect(
    user="louspringer",
    account='CFCLGQP-HQB93436',
    private_key_file='/home/lou/snow-keys/rsa_key.p8',
    warehouse="PRD_ALV_WAREHOUSE",
    database="PRD_ALV",        
    schema='QUERY_LINKEDIN',
    role='ALV_PROD_ROLE',
) as con:
    df = pd.read_sql("SELECT * FROM raw_jobs", con)
df

/tmp/ipykernel_39009/844797126.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM raw_jobs", con)


,JOB_PK,REQUEST_PK,JSON_RESPONSE,CREATED_AT


In [2]:
from snowflake.snowpark import Session

In [3]:
# >>> connection_parameters = {
# ...    "account": "<your snowflake account>",
# ...    "user": "<your snowflake user>",
# ...    "password": "<your snowflake password>",
# ...    "role": "<your snowflake role>",  # optional
# ...    "warehouse": "<your snowflake warehouse>",  # optional
# ...    "database": "<your snowflake database>",  # optional
# ...    "schema": "<your snowflake schema>",  # optional
# ...  }  

# >>> new_session = Session.builder.configs(connection_parameters).create()
connection_parameters = {
    "user":"louspringer",
    "account":'CFCLGQP-HQB93436',
    "private_key_file":'/home/lou/snow-keys/rsa_key.p8',
    "warehouse":"PRD_ALV_WAREHOUSE",
    "database":"PRD_ALV",        
    "schema":'QUERY_LINKEDIN',
    "role":'ALV_PROD_ROLE',
}

In [4]:
session = Session.builder.configs(connection_parameters).create()

In [5]:
# Assuming 'session' is already created and configured appropriately

# Create the table
session.sql("""
CREATE OR REPLACE TABLE sample_product_data (
    id INT,
    parent_id INT,
    category_id INT,
    name VARCHAR,
    serial_number VARCHAR,
    key INT,
    "3rd" INT
)
""").collect()


[Row(status='Table SAMPLE_PRODUCT_DATA successfully created.')]

In [6]:

# Insert data into the table
session.sql("""
INSERT INTO sample_product_data VALUES
    (1, 0, 5, 'Product 1', 'prod-1', 1, 10),
    (2, 1, 5, 'Product 1A', 'prod-1-A', 1, 20),
    (3, 1, 5, 'Product 1B', 'prod-1-B', 1, 30),
    (4, 0, 10, 'Product 2', 'prod-2', 2, 40),
    (5, 4, 10, 'Product 2A', 'prod-2-A', 2, 50),
    (6, 4, 10, 'Product 2B', 'prod-2-B', 2, 60),
    (7, 0, 20, 'Product 3', 'prod-3', 3, 70),
    (8, 7, 20, 'Product 3A', 'prod-3-A', 3, 80),
    (9, 7, 20, 'Product 3B', 'prod-3-B', 3, 90),
    (10, 0, 50, 'Product 4', 'prod-4', 4, 100),
    (11, 10, 50, 'Product 4A', 'prod-4-A', 4, 100),
    (12, 10, 50, 'Product 4B', 'prod-4-B', 4, 100)
""").collect()


[Row(number of rows inserted=12)]

In [7]:
df_table = session.table("sample_product_data")

In [8]:
df_table.toPandas()

,ID,PARENT_ID,CATEGORY_ID,NAME,SERIAL_NUMBER,KEY,3rd
0,1,0,5,Product 1,prod-1,1,10
1,2,1,5,Product 1A,prod-1-A,1,20
2,3,1,5,Product 1B,prod-1-B,1,30
3,4,0,10,Product 2,prod-2,2,40
4,5,4,10,Product 2A,prod-2-A,2,50
5,6,4,10,Product 2B,prod-2-B,2,60
6,7,0,20,Product 3,prod-3,3,70
7,8,7,20,Product 3A,prod-3-A,3,80
8,9,7,20,Product 3B,prod-3-B,3,90
9,10,0,50,Product 4,prod-4,4,100


In [21]:
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives import serialization

with open("/home/lou/snow-keys/rsa_key.p8", "rb") as key:
    p_key= serialization.load_pem_private_key(
        key.read(),
        password=None,
        backend=default_backend()
    )

pkb = p_key.private_bytes(
    encoding=serialization.Encoding.DER,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption())

engine = create_engine(URL(
    account='CFCLGQP-HQB93436',
    user='louspringer',
    ),
    connect_args={
        'private_key': pkb,
        },
    )

In [22]:
engine.execute("select * from prd_alv.query_linkedin.job_listings")

/tmp/ipykernel_132784/527860910.py:1: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  engine.execute("select * from prd_alv.query_linkedin.job_listings")


In [23]:
connection = engine.connect()

## JDBC Connectivity

In [9]:
# user="louspringer",
# account='CFCLGQP-HQB93436',
# private_key_file='/home/lou/snow-keys/rsa_key.p8',
# warehouse="PRD_ALV_WAREHOUSE",
# database="PRD_ALV",        
# schema='QUERY_LINKEDIN',
# role='ALV_PROD_ROLE',

import snowflake.connector

class SnowflakeJDBCExample:
    def __init__(self):
        self.account = "CFCLGQP-HQB93436"
        self.user = "louspringer"
        self.private_key_path = "/home/lou/snow-keys/rsa_key.p8"
        self.role = "ALV_PROD_ROLE"
        self.warehouse = "PRD_ALV_WAREHOUSE"
        self.database = "PRD_ALV"
        self.schema = "QUERY_LINKEDIN"

    def connect(self):
        try:
            # Establish the connection
            conn = snowflake.connector.connect(
                account=self.account,
                user=self.user,
                private_key_file=self.private_key_path,
                role=self.role,
                warehouse=self.warehouse,
                database=self.database,
                schema=self.schema
            )
            print("Connection successful!")
            # Use the connection (for example, execute a query)
            # cursor = conn.cursor()
            # cursor.execute("SELECT CURRENT_TIMESTAMP()")
            # for row in cursor:
            #     print(row)
        except Exception as e:
            print(f"An error occurred: {e}")

if __name__ == "__main__":
    example = SnowflakeJDBCExample()
    example.connect()


Connection successful!
